In [3]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import statsmodels.api as sm
import joblib

In [4]:
df = pd.read_csv("../input/stockpricepredictons/AXISBANK.NS.csv", usecols=["Date", "Open", "Adj Close", "High", "Low"], index_col='Date',parse_dates=True)
df = df.iloc[-800::]
sm.graphics.tsa.plot_pacf(df["Open"], lags=10)
plt.show()

In [5]:
train = df

In [6]:
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()

scaler.fit(train)
scaled_train = scaler.transform(train)
joblib.dump(scaler, 'AXISBANK_scaler.pkl') 

In [7]:
from keras.preprocessing.sequence import TimeseriesGenerator

In [8]:
# define generator
n_input = 1
n_features = 4
generator = TimeseriesGenerator(scaled_train, scaled_train, length=n_input, batch_size=1)

In [9]:
X,y = generator[0]
print(f'Given the Array: \n{X.flatten()}')
print(f'Predict this y: \n {y}')

In [10]:
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM

In [11]:
# define model
model = Sequential()
model.add(LSTM(50, activation='relu', input_shape=(n_input, n_features),return_sequences=True))
model.add(Dense(4))
model.compile(optimizer='adam', loss='mse')

In [12]:
# fit model
model.fit(generator,epochs=10)

In [13]:
first_eval_batch = scaled_train[-n_input:]
current_batch = first_eval_batch.reshape((1, n_input, n_features))
current_pred = model.predict(current_batch)[0]
print(current_pred)
fin_pred = scaler.inverse_transform(current_pred)
print(fin_pred)

In [14]:
model.save("AXISBANK_model.h5")